In [1]:
import pandas as pd
import os
import json
import tiktoken
import openai
import numpy as np
import redis
from redis.commands.search.field import TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt
from langchain.llms import AzureOpenAI

# Load environment variables
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = "https://domzisopenai.openai.azure.com/"
openai.api_key = "3673181a896c4997baba06bc82db5e6f"

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_ENCODING = 'cl100k_base'
EMBEDDING_CHUNK_SIZE = 8000
COMPLETION_MODEL = 'text-davinci-003'

REDIS_INDEX_NAME = 'domtistestindex'
VECTOR_FIELD_IN_REDIS='item_vector'
NUMBER_PRODUCTS_INDEX=1000
CHOSEN_EMB_MODEL = 'TextEmbeddingAda002'
REDIS_ADDR = 'redisdbtriguc3.westeurope.redisenterprise.cache.azure.net'
REDIS_PORT = 10000
REDIS_PASSWORD = 'hB+FWOy6MDlsL21bmaD1plSNwTa6Vd70TnID02P0r+0='


# initialize tiktoken for encoding text
encoding = tiktoken.get_encoding(EMBEDDING_ENCODING)

In [2]:
data = pd.read_excel('https://domenstorageacc.blob.core.windows.net/randomstuff/Copy%20of%20kb-ada-embeddings-full.xlsx', skiprows=0)
data.head()

,ID,L1,L2,L3,L3A,L4,SAME_L2_L3,COMPLEX_QUESTION,DUMMY_TAXONOMY_ID,EXTERNAL ODGOVOR (STRANKI),HTML EXTERNAL,INTERNAL NAVODILA ZAPOSLENI,HTML INTERNAL,KLJUČNE BESEDE,DRUGI NIVO (DA/NE),KDO,NASLOVI,embedding
0,0,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo imena/priimka?,1,Vsebina vprašanja: Sprememba kontaktnih podatk...,1,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"ime, priimek, osebni podatek, kontakt, zahteve...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN,"[-0.0038990620523691177, 0.02420891262590885, ..."
1,1,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo kontaktnega podatka?,1,Vsebina vprašanja: Sprememba kontaktnih podatk...,2,Za spremembo kontaktnih podatkov je potrebno o...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"osebni podatek, kontakt, zahtevek, sprememba",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN,"[0.0032689073123037815, 0.02642892673611641, 0..."
2,2,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo telefonske številke?,1,Vsebina vprašanja: Sprememba kontaktnih podatk...,3,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"telefonska številka, telefon, osebni podatek, ...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN,"[0.002097523771226406, 0.02316191792488098, 0...."
3,3,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo e-maila/e-poštnega naslova?,1,Vsebina vprašanja: Sprememba kontaktnih podatk...,4,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"e-mail, e-pošta, e-naslov, kontakt, zahtevek, ...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN,"[-0.00352742662653327, 0.01909773238003254, 0...."
4,4,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo osebnih podatkov?,1,Vsebina vprašanja: Sprememba kontaktnih podatk...,5,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"naslov, pošta, sprememba, osebni podatki, zaht...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN,"[0.003534401999786496, 0.023303886875510216, 0..."


In [3]:
Questions_array = data['COMPLEX_QUESTION'].values
ID_array = data['ID'].values
L1_array = data['L1'].values
L2_array = data['L2'].values
L3_array = data['L3'].values
L4_array = data['L4'].values
DUMMY_TAXONOMY_ID_array = data['DUMMY_TAXONOMY_ID'].values
EXTERNAL_ODGOVOR_STRANKI_array = data['EXTERNAL ODGOVOR (STRANKI)'].values


In [153]:
L4_string = L4_array.astype(str)

In [154]:
print(L4_string)

['Kako uredim spremembo imena/priimka?'
 'Kako uredim spremembo kontaktnega podatka?'
 'Kako uredim spremembo telefonske številke?'
 'Kako uredim spremembo e-maila/e-poštnega naslova?'
 'Kako uredim spremembo osebnih podatkov?'
 'Kako uredim spremembo naslova za prejemanje pošte?'
 'Kako uredim spremembo načina plačila?' 'Preverjanje zavarovanj stranke'
 'Preverjanje dolgov za stranko'
 'Preverjanje premoženjskega stanja za stranko'
 'Preverjanje premoženjskega stanja za podjetje'
 'Preverjanje dolgov za podjetje' 'Zanimanje za naroka'
 'Zahteva za pregled pogodb sklenjenih pri Zavarovalnici Triglav'
 'Prejem izvršbe' 'Preverjanje dolgov' 'Blokiran račun, urejanje terjatev'
 'Preplačana premija' 'Dvojni odtegljaj premije'
 'Kako lahko prekinem polico?' 'Ali je polica prekinjena? (Ne)'
 'Ali je polica prekinjena? (Da)' 'Informacije o opominu'
 'Kakšno je stanje dolga?' 'Kakšne so možnosti odplačila dolga?'
 'Kakšno je stanje obrokov/plačil?' 'Ali je račun poravnan/ plačan?'
 'Kakšne so 

In [4]:
db_embeddings = data['embedding'].values

In [5]:
print(Questions_array[0])
print(db_embeddings[0])

Vsebina vprašanja: Sprememba kontaktnih podatkov
Strankina vprašanja (Kaj zanima stranko): Kako uredim spremembo imena/priimka?
[-0.0038990620523691177, 0.02420891262590885, 0.008120152167975903, -0.013411510735750198, -0.015318420715630054, 0.017755728214979172, -0.010576402768492699, -0.0032581635750830173, -0.02894461527466774, -0.01845029927790165, 0.017023272812366486, -0.0011484079295769334, 0.0014388643903657794, 0.006806783843785524, 0.003267634892836213, 0.0087073789909482, 0.029954899102449417, -0.014383908361196518, 0.015179506503045559, -0.03720368072390556, -0.007867581211030483, 0.027151362970471382, 0.006203770637512207, 0.021481148898601532, -0.026898792013525963, -0.012710626237094402, 0.010551146231591702, -0.020912863314151764, 0.001131832948885858, -0.020256180316209793, -0.020243551582098007, 0.0005280308541841805, -0.015747791156172752, 0.00138992874417454, -0.01648024655878544, -0.006127999629825354, 0.0014609643258154392, -0.01879126951098442, 0.0291466731578111

In [6]:
len(Questions_array)

606

In [20]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text):
    return openai.Embedding.create(input=text, engine=EMBEDDING_MODEL)["data"][0]["embedding"]

In [174]:
# Connect to Redis - here change the host, port, password and ssl parameters to match your Redis Enterprise setup
r = redis.StrictRedis(host='redisdbtriguc3.westeurope.redisenterprise.cache.azure.net',
    port=10000, db=0, password='hB+FWOy6MDlsL21bmaD1plSNwTa6Vd70TnID02P0r+0=', ssl=True)

In [175]:
# Test the Redis connection
r.ping()

True

In [176]:
indexes = r.execute_command("FT._List")
# Print the keys
for index in indexes:
    print(index)


b'acs_emb_index'
b'domzis_index_simple'
b'triglav_c3'
b'domzis_index_complex'


In [177]:
keys = r.execute_command("KEYS *")
print("Keys count", len(keys))
keys

Keys count 1818


[b'triglav_c3:34',
 b'domzis_doc:575',
 b'domzis_doc_simple:137',
 b'triglav_c3:217',
 b'domzis_doc:501',
 b'triglav_c3:163',
 b'domzis_doc_simple:370',
 b'triglav_c3:535',
 b'domzis_doc:442',
 b'domzis_doc:181',
 b'domzis_doc_simple:449',
 b'domzis_doc_simple:199',
 b'triglav_c3:244',
 b'domzis_doc:320',
 b'domzis_doc_simple:496',
 b'domzis_doc:303',
 b'domzis_doc_simple:286',
 b'domzis_doc:499',
 b'domzis_doc:113',
 b'triglav_c3:403',
 b'domzis_doc:522',
 b'triglav_c3:578',
 b'triglav_c3:97',
 b'domzis_doc_simple:2',
 b'domzis_doc:407',
 b'triglav_c3:278',
 b'domzis_doc_simple:479',
 b'triglav_c3:92',
 b'domzis_doc:200',
 b'domzis_doc_simple:277',
 b'triglav_c3:181',
 b'domzis_doc:351',
 b'domzis_doc:234',
 b'domzis_doc_simple:198',
 b'domzis_doc_simple:53',
 b'triglav_c3:310',
 b'domzis_doc:523',
 b'domzis_doc:311',
 b'domzis_doc:15',
 b'domzis_doc_simple:250',
 b'domzis_doc_simple:247',
 b'domzis_doc:179',
 b'domzis_doc:541',
 b'domzis_doc_simple:430',
 b'triglav_c3:495',
 b'trigla

In [178]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://domzisopenai.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "3673181a896c4997baba06bc82db5e6f"

In [179]:
llm = AzureOpenAI(
    deployment_name="text-davinci-003",
    model_name="text-davinci-003",
)

In [180]:
email_from_excel = """
Spoštovani
Prosim, če mi lahko preverite eno informacijo. Pri plači na OŠ Bistrica, Tržič mi že na leta trgajo nekaj pod nazivom Zavarovalnica Triglav. Pred dnevi pa me je sodelavka, ki se je poškodovala vprašala, če vem kaj o temu, da smo kolektivno nezgodno zavarovani pri vaši zavarovalnici. Tudi njej trgajo enak znesek kot meni, pa še njenemu možu. Sem se pa spomnila, da sem pred leti res dobila izplačilo za poškodbo.V šoli imamo novo vodstvo, novo tajnico in računovodstvo in ne ve, kam naj pogleda. Mogoče vi lahko preverite v sistemu.
Zanima me zato, ker sem se tudi sama hudo poškodovala in imam trajne posledice in bi uveljavljala če sem zavarovana.
Moji podatki:
Marijana Zaplotnik Šlibar, roj. 30.5.1974 v Kranju, stanujoča Zvirče 10 (prej Kosarska ulica 9a) Tržič, davčna: 92231543, emšo: 3005974505234, kontaktna številka: 031305049
Prosim za odgovor.
Lep pozdrav
Marijana Z. Šlibar
"""

In [181]:
from langchain import PromptTemplate


template = """You are given an EMAIL, find a question that encapsulates the EMAIL.
EMAIL: {email}
QUESTION:
"""


In [182]:
prompt = PromptTemplate(
    input_variables=["email"],
    template=template,
)

prompt_with_input_values = prompt.format(email=email_from_excel)

response = llm(prompt_with_input_values)

In [192]:
print(prompt_with_input_values)
print("------------------")
print(response)

From the email determine what taxonomy it belongs to, choose from the taxonomy below given in the JSON format:
EMAIL: 
Spoštovani
Prosim, če mi lahko preverite eno informacijo. Pri plači na OŠ Bistrica, Tržič mi že na leta trgajo nekaj pod nazivom Zavarovalnica Triglav. Pred dnevi pa me je sodelavka, ki se je poškodovala vprašala, če vem kaj o temu, da smo kolektivno nezgodno zavarovani pri vaši zavarovalnici. Tudi njej trgajo enak znesek kot meni, pa še njenemu možu. Sem se pa spomnila, da sem pred leti res dobila izplačilo za poškodbo.V šoli imamo novo vodstvo, novo tajnico in računovodstvo in ne ve, kam naj pogleda. Mogoče vi lahko preverite v sistemu.
Zanima me zato, ker sem se tudi sama hudo poškodovala in imam trajne posledice in bi uveljavljala če sem zavarovana.
Moji podatki:
Marijana Zaplotnik Šlibar, roj. 30.5.1974 v Kranju, stanujoča Zvirče 10 (prej Kosarska ulica 9a) Tržič, davčna: 92231543, emšo: 3005974505234, kontaktna številka: 031305049
Prosim za odgovor.
Lep pozdrav
M

In [184]:
# expected found question: Kaj pomeni odprta zavarovalna doba?

q_em = get_embedding(response)
q_em = np.array(q_em, dtype=np.float32)

e_em = get_embedding(email_from_excel)
e_em = np.array(e_em, dtype=np.float32)

In [185]:
INDEX_NAME = "domzis_index_simple"

In [186]:
query = (
    Query("(@tag:{ openai })=>[KNN 5 @vector $vec as score]")
     .sort_by("score")
     .return_fields("content", "nivo3", "nivo3A", "tag", "score")
     .paging(0, 5)
     .dialect(2)
)

query_params = {"vec": q_em.tobytes()}
temp_json_question = r.ft(INDEX_NAME).search(query, query_params).docs

In [187]:
query = (
    Query("(@tag:{ openai })=>[KNN 5 @vector $vec as score]")
     .sort_by("score")
     .return_fields("content", "nivo3", "nivo3A", "tag", "score")
     .paging(0, 5)
     .dialect(2)
)

query_params = {"vec": e_em.tobytes()}
temp_json_email = r.ft(INDEX_NAME).search(query, query_params).docs

In [188]:
len(temp_json_question)
len(temp_json_email)

5

In [168]:
for doc in temp_json_email:
    print("Taxonomy:\n", doc['content'], "\nScore:", doc['score'], "\n")

print("--------------------------------------------------")

for doc in temp_json_question:
    print("Taxonomy:\n", doc['content'], "\nScore:", doc['score'], "\n")


Taxonomy:
 Vsebina vprašanja v vrsta storitve: Triglav komplet v Terjatve / plačila
Strankina vprašanja (Kaj zanima stranko): Avtomobil imam na lizing. Tudi zavarovanje imam urejeno preko lizing podjetja. Ali to zavarovanje lahko vključim v svoj Triglav komplet? 
Score: 0.107004582882 

Taxonomy:
 Vsebina vprašanja v vrsta storitve: Triglav komplet v Terjatve / plačila
Strankina vprašanja (Kaj zanima stranko): Preko spleta smo sklenili letno zavarovanje potovanj v tujino, ki prične veljati šele čez teden dni. Ali zavarovanje lahko vključim v Triglav komplet? 
Score: 0.111930668354 

Taxonomy:
 Vsebina vprašanja v vrsta storitve: Triglav komplet v Terjatve / plačila
Strankina vprašanja (Kaj zanima stranko): S koliko zavarovalnimi policami lahko sestavim svoj Triglav komplet? Kaj pa moj sin, ki bi se priključil kasneje? 
Score: 0.115045666695 

Taxonomy:
 Vsebina vprašanja v vrsta storitve: Zavarovanja vozil v Zavarovalne police
Ožje področje: Zavarovanje avtomobilske odgovornosti (obvez

In [189]:
template = """From the email determine what taxonomy it belongs to, choose from the taxonomy below given in the JSON format:
EMAIL: {email}
AVALABLE TAXONOMY: {taxonomy1} {taxonomy2}
DETERMINED TAXONOMY:
"""

In [190]:
prompt = PromptTemplate(
    input_variables=["email", "taxonomy1", "taxonomy2"],
    template=template,
)

prompt_with_input_values = prompt.format(email=email_from_excel, taxonomy1=temp_json_email, taxonomy2=temp_json_question)

response = llm(prompt_with_input_values)

In [191]:
print(prompt_with_input_values)
print("------------------")
print(response)

From the email determine what taxonomy it belongs to, choose from the taxonomy below given in the JSON format:
EMAIL: 
Spoštovani
Prosim, če mi lahko preverite eno informacijo. Pri plači na OŠ Bistrica, Tržič mi že na leta trgajo nekaj pod nazivom Zavarovalnica Triglav. Pred dnevi pa me je sodelavka, ki se je poškodovala vprašala, če vem kaj o temu, da smo kolektivno nezgodno zavarovani pri vaši zavarovalnici. Tudi njej trgajo enak znesek kot meni, pa še njenemu možu. Sem se pa spomnila, da sem pred leti res dobila izplačilo za poškodbo.V šoli imamo novo vodstvo, novo tajnico in računovodstvo in ne ve, kam naj pogleda. Mogoče vi lahko preverite v sistemu.
Zanima me zato, ker sem se tudi sama hudo poškodovala in imam trajne posledice in bi uveljavljala če sem zavarovana.
Moji podatki:
Marijana Zaplotnik Šlibar, roj. 30.5.1974 v Kranju, stanujoča Zvirče 10 (prej Kosarska ulica 9a) Tržič, davčna: 92231543, emšo: 3005974505234, kontaktna številka: 031305049
Prosim za odgovor.
Lep pozdrav
M